In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import nltk
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('portuguese')
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('portuguese')
import re
from unidecode import unidecode
from keras.regularizers import l2

In [ ]:
# Base original da B2W, sem cortes.
b2wCorpus = pd.read_csv("B2W-Reviews01.csv",";",usecols=['review_text','overall_rating'])
print(b2wCorpus.head)

In [ ]:
# função de pré-processamento
text_cleaning_re = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"
def preprocess(text, stem=False):
  text = unidecode(text)
  text = re.sub(text_cleaning_re, ' ', str(text).lower()).strip()
  text = re.sub("\d+", "", text)
  text = re.sub(r'(?:^| )\w(?:$| )', ' ', text).strip()
  tokens = []
  for token in text.split():
    if token not in stop_words:
      if stem:
        tokens.append(stemmer.stem(token))
      else:
        tokens.append(token)
  return " ".join(tokens)

In [ ]:
# pré-processamento
b2wCorpus.review_text = b2wCorpus.review_text.apply(lambda x: preprocess(x))

In [ ]:
print(b2wCorpus.head)

In [ ]:
b2wCorpus.overall_rating .value_counts().sort_index()

In [ ]:
Words = [len(linha.split()) for linha in b2wCorpus["review_text"]]
print(Words)
plt.hist(Words, len(Words))
plt.xlim(xmin=0, xmax = 100)
plt.show()